In [ ]:
%pip install git+https://github.com/bigscience-workshop/petals
%pip install --upgrade langchain

In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from transformers import AutoTokenizer
from petals import AutoDistributedModelForCausalLM
from langchain.llms.base import LLM

model_name = "petals-team/StableBeluga2"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, add_bos_token=False)
model = AutoDistributedModelForCausalLM.from_pretrained(model_name)
model = model.cuda()  # Move the model to GPU if available

print(model)


Nov 02 01:00:54.155 [INFO] Make sure you follow the LLaMA's terms of use: https://bit.ly/llama2-license for LLaMA 2, https://bit.ly/llama-license for LLaMA 1
Nov 02 01:00:54.155 [INFO] Using DHT prefix: StableBeluga2-hf
torch_shm_manager: error while loading shared libraries: libcupti.so.12: cannot open shared object file: No such file or directory


RuntimeError: no response from torch_shm_manager at "/home/biscuitbobby/Documents/Projects/semantic_kernel_test/venv/lib64/python3.11/site-packages/torch/bin/torch_shm_manager"

In [7]:
from langchain.llms.base import LLM
from transformers import AutoTokenizer
from petals import AutoDistributedModelForCausalLM

class CustomLLM(LLM):
    def __init__(self, model_name: str, **kwargs):
        super().__init__(**kwargs)
        object.__setattr__(self, "tokenizer", AutoTokenizer.from_pretrained(model_name, use_fast=False, add_bos_token=False))
        object.__setattr__(self, "model", AutoDistributedModelForCausalLM.from_pretrained(model_name))
        self.model.cuda()  # Ensure the model is on GPU

    def _call(self, text: str, stop=None, **kwargs) -> str:
        inputs = tokenizer(text, return_tensors="pt")["input_ids"].cuda()
        outputs = self.model.generate(inputs, max_new_tokens=150)
        decoded_output = (tokenizer.decode(outputs[0]))
        return decoded_output

    @property
    def _llm_type(self) -> str:
        return "CustomLLM"


In [8]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Initialize the custom LLM
llm = CustomLLM(model_name="petals-team/StableBeluga2")

# Define the prompt template
template = """
Question: {question}
Answer: Let's think step by step.
"""
prompt = PromptTemplate(template=template, input_variables=["question"])

# Create the LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm)

# Generate a response
question = "What is the capital of France?"
response = llm_chain.run(question)
print(response)


Nov 02 01:01:49.031 [INFO] Make sure you follow the LLaMA's terms of use: https://bit.ly/llama2-license for LLaMA 2, https://bit.ly/llama-license for LLaMA 1
Nov 02 01:01:49.032 [INFO] Using DHT prefix: StableBeluga2-hf
torch_shm_manager: error while loading shared libraries: libcupti.so.12: cannot open shared object file: No such file or directory


RuntimeError: no response from torch_shm_manager at "/home/biscuitbobby/Documents/Projects/semantic_kernel_test/venv/lib64/python3.11/site-packages/torch/bin/torch_shm_manager"